# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to divyarvshnkr@gmail.com and will expire on September 05, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1486902943.log


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features[:]]
#     print(features_sframe)
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
#     print(output_array)
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    #should return (normalized_features, norms)
    return (normalized_features, norms)

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix,weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [33]:
def get_ro(feature_mat,op,pred,wt):
    ro_i = []
    for i in xrange(len(wt)):
        feature_i = feature_mat[:,i]
        multiplier = (output - prediction) + wt[i]*feature_i
        ro_i_current = (feature_i * multiplier).sum()
        ro_i.append(ro_i_current)
        print ('ro_i for w[%s]: %e') % (i,ro_i_current)
    return ro_i

In [35]:
get_ro(simple_feature_matrix,output,prediction,weights)

ro_i for w[0]: 7.940030e+07
ro_i for w[1]: 8.793947e+07
ro_i for w[2]: 8.096670e+07


[79400300.034929156, 87939470.772991076, 80966698.675965652]

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [31]:
l1_penalty_values = np.logspace(8,8.375)

In [32]:
l1_penalty_values

array([  1.00000000e+08,   1.01777801e+08,   1.03587207e+08,
         1.05428781e+08,   1.07303094e+08,   1.09210729e+08,
         1.11152278e+08,   1.13128344e+08,   1.15139540e+08,
         1.17186491e+08,   1.19269833e+08,   1.21390213e+08,
         1.23548289e+08,   1.25744731e+08,   1.27980221e+08,
         1.30255454e+08,   1.32571137e+08,   1.34927987e+08,
         1.37326737e+08,   1.39768133e+08,   1.42252931e+08,
         1.44781905e+08,   1.47355838e+08,   1.49975531e+08,
         1.52641797e+08,   1.55355463e+08,   1.58117374e+08,
         1.60928385e+08,   1.63789371e+08,   1.66701219e+08,
         1.69664834e+08,   1.72681136e+08,   1.75751062e+08,
         1.78875566e+08,   1.82055616e+08,   1.85292202e+08,
         1.88586328e+08,   1.91939017e+08,   1.95351309e+08,
         1.98824266e+08,   2.02358965e+08,   2.05956503e+08,
         2.09617999e+08,   2.13344589e+08,   2.17137430e+08,
         2.20997701e+08,   2.24926599e+08,   2.28925345e+08,
         2.32995181e+08,

In [243]:
print 'w1 is not 0 and w2 is 0'
for i in xrange(len(l1_penalty_values)):
    l_by_2 = l1_penalty_values[i]/2.0
    val1 = (-l_by_2 < ro_i[1] and ro_i[1] < l_by_2)
    val2 = (-l_by_2 < ro_i[2] and ro_i[2] < l_by_2)
    if (val2 and not val1):
        print ('%e is in this range' % l1_penalty_values[i])

w1 is not 0 and w2 is 0
1.637894e+08 is in this range
1.667012e+08 is in this range
1.696648e+08 is in this range
1.726811e+08 is in this range
1.757511e+08 is in this range


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [39]:
for i in xrange(len(l1_penalty_values)):
    l_by_2 = l1_penalty_values[i]/2.0
    val1 = (-l_by_2 < ro_i[1] and ro_i[1] < l_by_2)
    val2 = (-l_by_2 < ro_i[2] and ro_i[2] < l_by_2)
    if (val1 and val2):
        print ('%e is in this range' % l1_penalty_values[i])

1.788756e+08 is in this range
1.820556e+08 is in this range
1.852922e+08 is in this range
1.885863e+08 is in this range
1.919390e+08 is in this range
1.953513e+08 is in this range
1.988243e+08 is in this range
2.023590e+08 is in this range
2.059565e+08 is in this range
2.096180e+08 is in this range
2.133446e+08 is in this range
2.171374e+08 is in this range
2.209977e+08 is in this range
2.249266e+08 is in this range
2.289253e+08 is in this range
2.329952e+08 is in this range
2.371374e+08 is in this range


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [87]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix=feature_matrix,weights=weights)
    feature_i = feature_matrix[:,i]
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_ii = ((feature_i)*(output - prediction + (weights[i] * feature_i))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_ii
    elif ro_ii < -l1_penalty/2.:
        new_weight_i = ro_ii + l1_penalty/2.
    elif ro_ii > l1_penalty/2.:
        new_weight_i = ro_ii - l1_penalty/2.
    else:
        new_weight_i = 0.
    return new_weight_i

To test the function, run the following cell:

In [88]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [208]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    d = len(initial_weights)
#     print d
    weights = initial_weights
    max_weights_change = tolerance
    while (max_weights_change >= tolerance):
        max_weights_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weights_change = abs(old_weights_i - weights[i])
            if weights_change > max_weights_change:
                max_weights_change = weights_change
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [209]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights_1e7 = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [210]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [211]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights_1e7, l1_penalty, tolerance)

In [212]:
weights

array([ 21624998.36636292,  63157246.78545421,         0.        ])

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [213]:
def getRSS(pred, op):
    errr = pred - op
    sqerrr = errr**2
    rss_s = sqerrr.sum()
    return rss_s

In [214]:
predd = predict_output(normalized_simple_feature_matrix,weights)
print getRSS(predd, output)

1.63049248148e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [215]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [216]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [217]:
(feature_matrix, output1) = get_numpy_data(train_data, all_features, my_output)
(normalized_feature_matrix, norms1) = normalize_features(feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [227]:
l1_penalty=1e7
tolerance=1.0
initial_weights_1e7=np.zeros(len(all_features) + 1)

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output1,
                                            initial_weights_1e7, l1_penalty, tolerance)
weights1e7_normalized = weights1e7 / norms1
print weights1e7_normalized
print weights1e7_normalized[3]

[  1.85285533e+05   0.00000000e+00   0.00000000e+00   1.61317456e+02
   0.00000000e+00   0.00000000e+00   2.87664700e+05   6.91937057e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
161.317456248


In [228]:
weights1e7

array([ 24429600.60933314,         0.        ,         0.        ,
        48389174.35227978,         0.        ,         0.        ,
         3317511.16271982,   7329961.9848964 ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [231]:
l1_penalty=1e8
tolerance=1.0
initial_weights_1e8=np.zeros(len(all_features) + 1)

weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output1,
                                            initial_weights_1e8, l1_penalty, tolerance)

weights1e8_normalized = weights1e8 / norms1
print weights1e8_normalized


[ 539366.62822135       0.               0.               0.               0.
       0.               0.               0.               0.               0.
       0.               0.               0.               0.        ]


In [232]:
weights1e8

array([ 71114625.75280938,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [233]:
l1_penalty=1e4
tolerance=5e5
initial_weights_1e4=np.zeros(len(all_features) + 1)

weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output1,
                                            initial_weights_1e4, l1_penalty, tolerance)
weights1e4_normalized = weights1e4 / norms1
print weights1e4_normalized

[  5.89912924e+05  -4.97435039e+04   5.17044250e+04   3.07261390e+02
  -3.67765574e-01  -4.32048893e+04   5.63119400e+05   6.66940353e+04
   8.99767715e+03   1.80569342e+04  -5.60846894e+01  -7.88384489e+01
  -3.21603081e+02   5.18531810e+01]


In [234]:
weights1e4

array([ 77779073.91265215, -22884012.25023361,  15348487.08089997,
        92166869.69883084,  -2139328.0824278 ,  -8818455.54409496,
         6494209.73310655,   7065162.05053197,   4119079.21006765,
        18436483.52618778, -14566678.54514349,  -5528348.75179429,
       -83591746.20730527,   2784276.46012858])

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [235]:
print weights1e7_normalized[3]

161.317456248


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [236]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [237]:
def get_rss1(features_matrix, weights, output):
    print weights
    prediction = predict_output(features_matrix, weights)
    residuals = prediction - output
    return (residuals * residuals).sum()

In [240]:
rss_weights1e4 = get_rss1(test_feature_matrix, weights1e4_normalized, test_output)
print rss_weights1e4


[  5.89912924e+05  -4.97435039e+04   5.17044250e+04   3.07261390e+02
  -3.67765574e-01  -4.32048893e+04   5.63119400e+05   6.66940353e+04
   8.99767715e+03   1.80569342e+04  -5.60846894e+01  -7.88384489e+01
  -3.21603081e+02   5.18531810e+01]
2.2778100476e+14


In [241]:
rss_weights1e7 = get_rss1(test_feature_matrix, weights1e7_normalized, test_output)
print rss_weights1e7

[  1.85285533e+05   0.00000000e+00   0.00000000e+00   1.61317456e+02
   0.00000000e+00   0.00000000e+00   2.87664700e+05   6.91937057e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
2.75962079909e+14


In [242]:
rss_weights1e8 = get_rss1(test_feature_matrix, weights1e8_normalized, test_output)
print rss_weights1e8


[ 539366.62822135       0.               0.               0.               0.
       0.               0.               0.               0.               0.
       0.               0.               0.               0.        ]
5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?

A: 1e4

https://github.com/chengzh2008/machine-learning-regression-python/blob/master/week5/week-5-lasso-assignment-2-blank.ipynb